## Dyanmic Attributes and Properties

### Data Wrangling

In [1]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = f'Downloading {URL} to {JSON}'
        warnings.warn(msg)
        
        with urlopen(URL) as remote, open(JSON, 'wb', encoding="utf8") as local:
            local.write(remote.read())
    
    with open(JSON, encoding="utf8") as fp:
        return json.load(fp)

In [2]:
feed = load()

In [3]:
feed['Schedule'].keys()

dict_keys(['conferences', 'events', 'speakers', 'venues'])

In [4]:
for k,v in feed['Schedule'].items():
    print(f'{len(v):3} {k}')

  1 conferences
494 events
357 speakers
 53 venues


In [5]:
feed['Schedule']['conferences']

[{'serial': 115}]

In [6]:
feed['Schedule']['events'][0]

{'serial': 33451,
 'name': 'Migrating to the Web Using Dart and Polymer - A Guide for Legacy OOP Developers',
 'event_type': '40-minute conference session',
 'time_start': '2014-07-23 17:00:00',
 'time_stop': '2014-07-23 17:40:00',
 'venue_serial': 1458,
 'description': 'The web development platform is massive. With tons of libraries, frameworks and concepts out there, it might be daunting for the &#39;legacy&#39; developer to jump into it.\r\n\r\nIn this presentation we will introduce Google Dart &amp; Polymer. Two hot technologies that work in harmony to create powerful web applications using concepts familiar to OOP developers.',
 'website_url': 'https://conferences.oreilly.com/oscon/oscon2014/public/schedule/detail/33451',
 'speakers': [149868],
 'categories': ['Emerging Languages']}

Accessing the values using above syntax is difficult. Following class can be used to access the values similar to **JavaScript** *(feed.Schedule.events)*

`__getattr__` method can be used to retrive the attributes using `dot and letters`. This method is invoked only if the usual process fails to retrieve an attribte

In [7]:
from collections import abc
from keyword import iskeyword

class FrozenJSON:
    def __init__(self, mapping):
        self.__data = {}
        for k,v in mapping.items():
            if iskeyword(k):
                k += '_'
            self.__data[k] = v
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
        
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

1. Build a dictionary from *mapping* argument. It also checks if the key is python keyword. If yes then it append an **underscore _** to it.

2. `__getattr__` is called when there is no attribute of *name* present.

    a. If *name* matches an attribute of instance `__data`, return that.
    
    b. Else fetch the item with the key name from `self.__data` and return the result of `build()` method on that.


3. `build()` is an alternative constructor built using `classmethod`.

    a. If obj is mapping, build a `FrozenJSON` with it.
    
    b. If it's mutable sequence, then build list.
    
    c. If not either of these, return the obj as it is.

In [8]:
raw_feed = load()
feed = FrozenJSON(raw_feed)

In [9]:
len(feed.Schedule.speakers)

357

In [10]:
for k,v in feed.Schedule.items():
    print(f'{len(v):3} {k}')

  1 conferences
494 events
357 speakers
 53 venues


In [11]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [12]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [13]:
grad.class_

1982

### Object creation with `__new__`

Special method that actually constructs an instance is `__new__`. It's a classmethod without a decorated and it must return an instance.

This instance will in turn be passed as first argument *self* of **\_\_init__()**.

**\_\_init__()** is actaully forbidden from returning anything, it's just an `initializer`. The real constructor is `__new__`. The `__new__` method can also return an instance of different class

In [14]:
from collections import abc
from keyword import iskeyword

class FrozenJSON:
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls[item] for item in arg]
        else:
            return arg
    
    def __init__(self, mapping):
        self.__data = {}
        for k,v in mapping.items():
            if iskeyword(k):
                k += '_'
            self.__data[k] = v
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])
        

Here `__new__` method gets the class as first argument. 

Expression `super().__new__(cls)` `object.__new__(FrozenJSON)`. Hence the `__class__` attribute of new instance will hold reference to **FrozenJSON**

In [15]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [16]:
grad.class_

1982

### Using a property for attribute validation

Simple class for selling organic food in bulk

In [17]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [18]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [19]:
raisins.weight = -20
raisins.subtotal()

-139.0

If this code is already in production, we can fix this by using `property` 

In [20]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self._weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self._weight = value
        else:
            raise ValueError('Weight must be greater than 0')

In [21]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [22]:
raisins.weight = -20
raisins.subtotal()

ValueError: Weight must be greater than 0

#### Coding a Property Factory

In [23]:
def quantity(storage_name):
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('Value must be greater than 0')
    
    return property(qty_getter, qty_setter)

class LineItem:
    
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [24]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [25]:
vars(nutmeg)

{'description': 'Moluccan nutmeg', 'weight': 8, 'price': 13.95}

*storage_name* determines where the data for each property is stored. For *weight* argument, storage name will be *'weight'*.

*instance* refers to **LineItem** instance where the attribute is stored. *qty_getter* references *storage_name*, so it will be preserved in the closure of this function. Value is directly retrieved from *instance.\_\_dict__* to bypass the property and avoid infinite recursion.

The value is stored directly in the *instance.\_\_dict__*, again bypassing the property.

Then function builds the custom property object and returns

#### Handling Attribute Deletion

In [26]:
class BlackKnight:
    
    def __init__(self):
        self.members = ['an arm', 'another arm', 'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.", "It's just a flesh wound.", "I'm invincible!", 
                        "All right, we'll call it a draw."]
    
    @property
    def member(self):
        print('Next member is:')
        return self.members[0]
    
    @member.deleter
    def member(self):
        print(f'BLACK KNIGHT (loses {self.members.pop(0)})\n-- {self.phrases.pop(0)})') 

In [27]:
knight = BlackKnight()
knight.member

Next member is:


'an arm'

In [28]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.)


In [29]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.)


In [30]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!)


In [31]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.)


### Essential attributes and Functions for attribute handling

#### Special Attributes that affect attribute handling

Behaviour of many functions and special methods depends on three special attributes - 

\_\_class__ :
    
    A reference to an object's class. 

\_\_dict__ :

    A mapping that stores the writable attributes of an object or class. 

\_\_slots__ :

    An attribute that may be defined in class to limit the attributes in its instances have. 
    It is a tuple of strings. If __dict__ is not in __slots__ , then the instances os that class wont have dict of 
    their own
    
#### Built in functions for attribute handling

dir([object]) :
    
    Lists most of the attributes of object. If optional argument (object) is not given, it returns the names in 
    current scope
    
getattr(object, name[, default]) :

    Gets the attribute identified by `name` string of an object. If no such attribute exist, it raises 
    AttributeError or returns default if given.
    
hasattr(object, name) :

    Returns True if named attribute axists in object. 

setattr(object, name, value) :

    Assigns the value to named attribute if object allows. This may create new attribute or override the existing
    
vars([object]) :

    Returns the dictionary of object. Without argument returns a dict representing local scope

## Attribute Descriptors

Descriptors are way of reusing same access logic in multiple attriubtes.

Descriptor is a class that implements a protocol consisting of `__get__`,`__set__` and `__delete__` methods. 

`Descriptor class` - A class implementing descritor protocol

`Managed class` - The class where the descriptor instances are declared as class attributes

`Descriptor Instance` - Each instance of a descriptor class, declared as a class attribute of the managed class. 

`Managed Instance` - One instance of managed class

`Storage Attribute` - An attribute of managed instance that will hold the value of managed attribute for that particular instance.

`Managed Attribute` - A public attribute in managed class that will be handled by descriptor class, with the values stored in storage attributes.

In [32]:
class Quantity: #Descriptor Class
    
    def __init__(self, storage_name):
        self.storage_name = storage_name
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('Value must be greater than 0')
            
class LineItem: #Managed Class
    
    weight = Quantity('weight') #Managed Attribute
    price = Quantity('price') #Managed Attribute
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [33]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price #Storage Attributes
#nutmeg is managed instance

(8, 13.95)

In [34]:
truffle = LineItem('White truffle', 100, 0)

ValueError: Value must be greater than 0

**In `__set__` method *self* is descriptor instance and *instance* is managed instance. Descriptors managing instance attributes should store values in managed instances.**

### Automatic Storage Attribute names

The drawback of earlier code is the need to repeat the names of attributes like `weight = Quantity('weight')`.

One workaround for this is 

In [39]:
class Quantity:
    
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = f'_{prefix}#{index}'
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('Value must be greater than 0')
            
class LineItem: #Managed Class
    
    weight = Quantity() #Managed Attribute
    price = Quantity() #Managed Attribute
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

`__counter` is class attribute of *Quantity* counting the number of *Qunatity* instances

*cls* is a reference to class *Quantity*

*storage_name* for each descriptor instance is unique because it is built from the descritor class name and counter value

`__get__` method is required the name of managed attribute is not same as *storage_name*

In [41]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)
coconuts.weight, coconuts.price
#Storage attributes

(20, 17.95)

In [37]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

*owner* argument in `__get__` method is reference to managed class and it's handy when descriptor is used to get attributes from the class.

If we try to access the managed attribute using class name we get AttributeError. 

In [38]:
LineItem.weight

AttributeError: 'NoneType' object has no attribute '_Quantity#0'

If managed attribute is accessed via classname ,`__get__` received **None** as a value for *instance* argument.

It's a good practice to make `__get__` method return the descriptor instance when the managed attribute is accessed through class

In [42]:
class Quantity:
    
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = f'_{prefix}#{index}'
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('Value must be greater than 0')
            
class LineItem: #Managed Class
    
    weight = Quantity() #Storage Attribute
    price = Quantity() #Storage Attribute
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [43]:
LineItem.price

In [44]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

Usually we define descriptor in differnt module which can be used across the application

### Property factory vs Descriptor class

In [46]:
def quantity():
    try:
        quantity.counter += 1
    except AttributeError:
        quantity.counter = 0
        
    storage_name = f'_quantity:{quantity.counter}'
        
    def qty_getter(instance):
        return getattr(instance, storage_name)
    
    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('Value must be greater than 0')
            
    return property(qty_getter, qty_setter)
    

class LineItem: #Managed Class
    
    weight = quantity() #Storage Attribute
    price = quantity() #Storage Attribute
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [47]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

A descriptor class can be extended by subclassing.

Property factory code does not depend on storage object relationships like used in descriptor class (*instance*, *self*)